In [4]:
import pandas as pd
import MySQLdb
import os
import time

In [199]:
db = MySQLdb.connect("47.100.14.137","hyps44413","Nanhua")
db.set_character_set('gbk')

In [2]:
db = MySQLdb.connect("localhost","root","123456")
db.set_character_set("gbk")

In [156]:
db.close()

# 單獨品種日資料

In [ ]:
DATA_DIR = "../CL Daily Data WTI原油期货-所有资料"
DATA_DIR = "../all IOE"

#SQL_INSERT("Iron_original_data",DATA_DIR)
#SQL_INSERT("WTI_Oil_original_data",DATA_DIR)

In [102]:
def SQL_INSERT(dbname,DATA_DIR):
    for filename in os.listdir(DATA_DIR):      
        cursor = db.cursor()
        sql = "CREATE DATABASE IF NOT EXISTS " + dbname
        cursor.execute(sql)

        contract = filename.split(".xlsx")[0]

        sql = "CREATE TABLE IF NOT EXISTS " dbname + "." + contract + \
              " (Date DATE PRIMARY KEY,\
                Open FLOAT(5),\
                High FLOAT(5),\
                Low FLOAT(5),\
                Close FLOAT(5),\
                Volume FLOAT(5),\
                OI FLOAT(5))"
        cursor.execute(sql)


        data = pd.read_excel(DATA_DIR+"/"+filename,header=None)
        data = data.fillna("NULL")


        for i in range(8,len(data)):
            cursor = db.cursor()
            sql = "insert into " + dbname + "." + contract + " values ('"+ data.iloc[i][0].strftime("%Y-%m-%d") + "',"\
                    + str(data.iloc[i][1]) + ","\
                    + str(data.iloc[i][2]) + ","\
                    + str(data.iloc[i][3]) + ","\
                    + str(data.iloc[i][4]) + ","\
                    + str(data.iloc[i][5]) + ","\
                    + str(data.iloc[i][6]) + ")"
            cursor.execute(sql)
            db.commit()
        cursor.close()

SyntaxError: invalid syntax (<ipython-input-102-155356abac9b>, line 9)

# 合約分開

In [153]:
DATA_DIR = '../ALL CONCRACT/鄭商所'
for filename in os.listdir(DATA_DIR):
    
    data = pd.read_excel(DATA_DIR + "/" + filename,header=None)
    
    folderName = filename.split(".xlsx")[0]
    
    name_column = 1
    column = 0
    next_column = 8
    while next_column < len(data.loc[0]):
        name = data.loc[0][name_column]
        split_data = data.iloc[8::,column:next_column]

        #刪除所有列都是空值的行
        split_data = split_data.dropna(how='all')

        #strip()刪除字串空白
        head=['Date','PX_OPEN','PX_HIGH','PX_LOW','PX_LAST','PX_SETTLE','VOLUME','OPEN_INT']

        split_data.to_csv('../ALL CONCRACT/CZCE/'+folderName+"/"+name.strip()+'.csv',header=head,index=None)
        column += 9
        next_column += 9
        name_column += 9

In [24]:
data = pd.read_excel('../日資料/123.xlsx',header=None)
    
name_column = 1
column = 0
next_column = 8
while next_column <= len(data.loc[0]):
    name = data.loc[0][name_column]
    split_data = data.iloc[8::,column:next_column]

    #刪除所有列都是空值的行
    split_data = split_data.dropna(how='all')

    #strip()刪除字串空白
    head=['Date','PX_OPEN','PX_HIGH','PX_LOW','PX_LAST','PX_SETTLE','VOLUME','OPEN_INT']

    split_data.to_csv('../日資料/' + name.strip()+'.csv',header=head,index=None)
    column += 9
    next_column += 9
    name_column += 9

# 所有交易所日資料

In [157]:
FOLDER_DIR = '../ALL CONCRACT/CZCE'
for foldername in os.listdir(FOLDER_DIR):
    cursor = db.cursor()
    
    databaseName = foldername + "_day_data"
    
    sql = "create database if not exists " + databaseName
    cursor.execute(sql)
    
    DATA_DIR = FOLDER_DIR + "/" + foldername
    for filename in os.listdir(DATA_DIR):
        
        name = filename.split('.csv')[0]
        table_name = name.split(' ')[0]
        
        data = pd.read_csv(DATA_DIR + "/" + filename,header=0)
        data['Date'] = pd.to_datetime(data['Date'])

        data = data.where((pd.notnull(data)), None)
        
        sql = "create table if not exists "+ databaseName + "." + table_name + \
                  " (Date DATE PRIMARY KEY,\
                    Open FLOAT(5),\
                    High  FLOAT(5),\
                    Low  FLOAT(5),\
                    Close  FLOAT(5),\
                    Settle  FLOAT(5),\
                    Volume  FLOAT(5),\
                    OI  FLOAT(5))"
        cursor.execute(sql)
        
        if len(data) == 0:
            continue
        
        values = []
        for i in range(len(data)):
            values.append((data.iloc[i][0],data.iloc[i][1],data.iloc[i][2],data.iloc[i][3],data.iloc[i][4],
                           data.iloc[i][5],data.iloc[i][6],data.iloc[i][7]))
        sql = "insert into " + databaseName + "." + table_name + " values (%s,%s,%s,%s,%s,%s,%s,%s)"
        cursor.executemany(sql,values)
        db.commit()   
    cursor.close()

In [10]:
db = MySQLdb.connect("47.100.14.137","hyps44413","Nanhua")
db.set_character_set('gbk')

In [21]:
db = MySQLdb.connect("localhost","root","123456")
db.set_character_set("gbk")

In [11]:
cursor = db.cursor()
    
databaseName = "au_day_data"
table_name = "AUAM08"
        
data = pd.read_excel('../日資料/AUAM08.xlsx',header=0)
data['Dates'] = pd.to_datetime(data['Dates'])

data = data.where((pd.notnull(data)), None)
        
sql = "create table if not exists "+ databaseName + "." + table_name + \
        " (Date DATE PRIMARY KEY,\
        Open FLOAT(5),\
        High  FLOAT(5),\
        Low  FLOAT(5),\
        Close  FLOAT(5),\
        Settle  FLOAT(5),\
        Volume  FLOAT(5),\
        OI  FLOAT(5))"
cursor.execute(sql)

values = []
for i in range(len(data)):
     values.append((data.iloc[i][0],data.iloc[i][1],data.iloc[i][2],data.iloc[i][3],data.iloc[i][4],
                     data.iloc[i][5],data.iloc[i][6],data.iloc[i][7]))
sql = "insert into " + databaseName + "." + table_name + " values (%s,%s,%s,%s,%s,%s,%s,%s)"
cursor.executemany(sql,values)
db.commit()   
cursor.close()

# 石油相關資料

In [5]:
db_list = {'XB':'NYMEX_XB','CL':'NYMEX_CL','HO':'NYMEX_HO','NG':'NYMEX_NG','LCO':'ICEUK_LCO','LGO':'ICEUK_LGO'}

In [15]:
db_list['LCO']

'ICEUK_LCO'

In [29]:
DATA_DIR = '../DOC HO'
cursor = db.cursor()

for file in os.listdir(DATA_DIR):
    name = file.split(".xlsx")[0]
    data = pd.read_excel(DATA_DIR+"/"+file)
    data = data.dropna(thresh=2)

    dbname = db_list[name] + "_W"


    ##重要!!!!!!!!!!!!!!!!!!!!!!!!
    #executemany 空值一定要改成None!!
    data = data.where((pd.notnull(data)), None)

    # date = pd.read_excel("../20171120 所有换约日期.xlsx")
    # contract_date = pd.to_datetime(date['XB期货连续合约换约日期'])
    # change_contract = pd.DataFrame()

    # #分開合約
    # change_contract['Date'] = data.iloc[9::,0]        

    # #增加一列全0
    # change_contract['change_contract'] = 0  

    # #升序排序跟重置index
    # change_contract = change_contract.sort_index(axis=0,ascending=False)
    # change_contract = change_contract.reset_index(drop=True)

    # #換約日期的下一次交易日change_contract改為1
    # for i in range(len(contract_date)):
    #     index = change_contract[change_contract['Date'] == contract_date.loc[i]].index.tolist()
    #     if(len(index) > 0):
    #         change_contract.iloc[index[0]+1,1] = 1

    # change_contract['Date'] = change_contract['Date'].apply(lambda x: x.strftime("%Y-%m-%d"))


    # values = []
    # for i in range(len(change_contract)):
    #     values.append(tuple(change_contract.loc[i]))

    # sql = "insert into " + dbname + ".change_contract values(%s,%s)"
    # cursor.executemany(sql,values)
    # db.commit()


    table_list = ['Open','High','Low','Close','Volume','OI']
    index = 1
    try:
        for table in table_list:
            print(table)
            values=[]
            for i in range(8,19):
                values.append([data.iloc[i][0],data.iloc[i][index],data.iloc[i][index+6],data.iloc[i][index+12],data.iloc[i][index+18],data.iloc[i][index+24],
                                data.iloc[i][index+30],data.iloc[i][index+36],data.iloc[i][index+42],data.iloc[i][index+48],data.iloc[i][index+54],
                                data.iloc[i][index+60],data.iloc[i][index+66]])

            cursor.executemany("INSERT INTO " + dbname + "." + table + " VALUES(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)".format(),values)
            db.commit()

            index+=1
    except Exception as e:
        print(e)

cursor.close()

Open
(1062, "Duplicate entry '2017-12-26' for key 'PRIMARY'")
Open
High
Low
Close
Volume
OI
Open
(1062, "Duplicate entry '2017-12-26' for key 'PRIMARY'")


In [26]:
db = MySQLdb.connect("47.100.14.137","hyps44413","Nanhua")
db.set_character_set('gbk')

In [24]:
db = MySQLdb.connect("localhost","root","123456")
db.set_character_set("gbk")

In [36]:
table_list = ['Open','High','Low','Close','Volume','OI']

cursor = db.cursor()
sql = "CREATE DATABASE IF NOT EXISTS NYMEX_XB_W"
cursor.execute(sql)

sql = "CREATE TABLE IF NOT EXISTS NYMEX_XB_W.change_contract " +\
      "(Date DATE,\
        change_day INT(2))"
cursor.execute(sql)

for i in table_list:
    
    sql = "CREATE TABLE IF NOT EXISTS NYMEX_XB_W."+ i + \
        " (Date DATE PRIMARY KEY,\
        XB1 FLOAT(5) DEFAULT NULL,\
        XB2 FLOAT(5) DEFAULT NULL,\
        XB3 FLOAT(5) DEFAULT NULL,\
        XB4 FLOAT(5) DEFAULT NULL,\
        XB5 FLOAT(5) DEFAULT NULL,\
        XB6 FLOAT(5) DEFAULT NULL,\
        XB7 FLOAT(5) DEFAULT NULL,\
        XB8 FLOAT(5) DEFAULT NULL,\
        XB9 FLOAT(5) DEFAULT NULL,\
        XB10 FLOAT(5) DEFAULT NULL,\
        XB11 FLOAT(5) DEFAULT NULL,\
        XB12 FLOAT(5)) CHARSET=utf8"        
    cursor.execute(sql)